In [18]:
import os, pickle
import Bio.PDB
import numpy as np
from Bio.PDB import PDBParser
from Bio.PDB import Superimposer
from Bio.PDB.Atom import *
from Bio.PDB.Residue import *
from Bio.PDB.Chain import *
from Bio.PDB.Model import *
from Bio.PDB.Structure import *
# from Bio.PDB.Vector import *
from Bio.PDB.Entity import*
import math
from Bio.PDB.vectors import Vector

from Bio import SeqIO
from Bio.PDB.DSSP import DSSP
from collections import defaultdict
from PeptideBuilder import Geometry
import PeptideBuilder
import warnings
warnings.filterwarnings("ignore")



#atom_dict = {'C':0, 'N':1, 'O':2, 'S':3}
atom_dict = {"N":0, "CA":1, "C":2, "O":3, "CB":4, "OG":5, "CG":6, "CD1":7, "CD2":8, "CE1":9, "CE2":10, "CZ":11, 
             "OD1":12, "ND2":13, "CG1":14, "CG2":15, "CD":16, "CE":17, "NZ":18, "OD2":19, "OE1":20, "NE2":21, 
             "OE2":22, "OH":23, "NE":24, "NH1":25, "NH2":26, "OG1":27, "SD":28, "ND1":29, "SG":30, "NE1":31, 
             "CE3":32, "CZ2":33, "CZ3":34, "CH2":35, "OXT":36}

RESIDUE_TYPES = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y','X']
parser = Bio.PDB.PDBParser(QUIET = True)

In [35]:
"""
3 bond lengths centered around residue angle frame: Ca-C, N-Ca, C_prev-N"""
def get_backbone_angles_and_bond_lengths(file): 
    #load structure
    structure = parser.get_structure("input", file)
    C_prev = None
    N_prev = None
    CA_prev = None
    #initialize arrays to store angles and planes
    residue_angles = {
        'phi': [],
        'psi': [],
        'omega': [],
        'planes': [],
        'CA_C_N_angle': [],
        'N_CA_C_angle': [],
        'C_N_Ca_angle': [],
        'CA_N_length': [],
        'CA_C_length': [],
        'N_C_length': []
    }
    #iterate over residues
    for res in structure.get_residues():
        #check if this is the first residue
        if C_prev is None:
            C_prev = res['C'].get_vector()
            N_prev = res['N'].get_vector()
            CA_prev = res['CA'].get_vector()
            continue
        #if it isn't we can get all the bond lengths and angles:
        else:
            try:
                print(res['CA'].get_vector())
                #get current residue vectors
                N = res['N'].get_vector()
                CA = res['CA'].get_vector()
                C = res['C'].get_vector()
                O = res['O'].get_vector() #just keep if needed
                print(C, N, CA, O)
            except:
                continue
            
            #calculate dihedral angles of curr residue using the reference atoms of previous C, N, CA
            residue_angles['phi'].append(math.radians(Bio.PDB.calc_dihedral(C_prev, N, CA, C)))
            residue_angles['psi'].append(math.radians(Bio.PDB.calc_dihedral(N_prev, CA_prev, C, N)))
            residue_angles['omega'].append(math.radians(Bio.PDB.calc_dihedral(CA_prev, C_prev, N, CA)))
            #calculate the plane
            residue_angles['planes'].append([N, CA, C])
            
            
            #calculate three angles among N, CA, and C
            CA_C_N_angle =math.radians(Bio.PDB.calc_angle(CA_prev, C_prev, N))
            N_CA_C_angle = math.radians(Bio.PDB.calc_angle(N, CA, C))
            C_N_Ca_angle = math.radians(Bio.PDB.calc_angle(C_prev, N, CA))   
            residue_angles['CA_C_N_angle'].append(CA_C_N_angle)
            residue_angles['N_CA_C_angle'].append(N_CA_C_angle)
            residue_angles['C_N_Ca_angle'].append(C_N_Ca_angle)
            
            #get the 3 bond lengths 
            CA_N_length = CA - N
            CA_C_length = CA - C
            N_C_length = N - C_prev #this is the peptide bond, so between two residue
            residue_angles['CA_N_length'].append(CA_N_length)
            residue_angles['CA_C_length'].append(CA_C_length)
            residue_angles['N_C_length'].append(N_C_length)
        
            
            #update the prev values by these new ones 
            C_prev = C
            N_prev = N
            CA_prev = CA
            
        #convert to numpy arrays
        phi_angles = np.vstack(residue_angles['phi'])
        psi_angles = np.vstack(residue_angles['psi'])
        omega_angles = np.vstack(residue_angles['omega'])
        planes = np.vstack(residue_angles['planes'])
        CA_C_N_angle = np.vstack(residue_angles['CA_C_N_angle'])
        N_CA_C_angle = np.vstack(residue_angles['N_CA_C_angle'])
        C_N_Ca_angle = np.vstack(residue_angles['C_N_Ca_angle'])
        CA_N_length = np.vstack(residue_angles['CA_N_length'])
        CA_C_length = np.vstack(residue_angles['CA_C_length'])
        N_C_length = np.vstack(residue_angles['N_C_length'])
        
        return phi_angles, psi_angles, omega_angles, planes, CA_C_N_angle, N_CA_C_angle, C_N_Ca_angle, CA_N_length, CA_C_length, N_C_length

In [ ]:
get_backbone_angles_and_bond_lengths("/work/slusky/s300y051/relax/test/relax/A0A009IHW8_relaxed_0001.pdb")

In [21]:
structure = parser.get_structure("input", "/work/slusky/s300y051/relax/test/relax/A0A009IHW8_relaxed_0001.pdb")
C_prev = None
N_prev = None
CA_prev = None
#initialize arrays to store angles and planes
residue_angles = {
    'phi': [],
    'psi': [],
    'omega': [],
    'planes': [],
    'CA_C_N_angle': [],
    'N_CA_C_angle': [],
    'C_N_Ca_angle': []
}